### Integrazione di Fondamenti di Scienza dei Dati
# Un prototipo per un'applicazione web in Flask per la raccolta di dati
**Dario Gaudiuso**

# Introduzione
L'utilizzo di un'applicazione web rappresenta una comoda soluzione per la raccolta dei dati a inserimento manuale da parte dell'utente, come per il caso dell'azienda analizzato nel progetto d'esame. La compilazione manuale, infatti, porta con sé tutto un bagaglio di errori ben noti che le soluzioni _naive_, come per esempio l'uso di Google Form, non sono in grado di individuare: in questo modo, i dati errati finiscono nel dataset, delegando la gestione dell'errore a chi farà uso dei dati raccolti. L'uso di un'applicazione web, invece, permette di mettere in atto diverse strategie per riconoscere gli errori di inserimento già nel tentativo di aggiunta di un nuovo record, garantendo che nel dataset non arrivino dati palesemente anomali.

Scopo del progetto è esplorare e sperimentare due tecnologie python per lo sviluppo di applicazioni web: Flask, un micro-framework per applicazioni web, e SQLAlchemy come ORM per interfacciarsi con la base di dati. Utilizzando questi strumenti verrà sviluppato un prototipo per un'applicazione web che consenta l'autenticazione dell'utente che sta effettuando l'inserimento e l'inserimento dei dati nel database.

# Funzionalità

L'applicazione permette l'inserimento nel database di mercati e corrispettivi. Solo gli utenti autenticati possono inserire corrispettivi e solo un utente con privilegi di admin può inserire o modificare i mercati.

## Errori e strategie di prevenzione

# Struttura del progetto

## Database e ORM
Come DBMS abbiamo scelto di utilizzare SQLite perché è il più semplice da configurare e mette a disposizione tutte le funzionalità che servono. Fortunatamente, Flask è dotato di una libreria `Flask-SQLAlchemy` che semplifica di molto la configurazione dell'ORM.

L'inizializzazione del database è effettuata nel file `database.py`, dove si trovano anche i modelli per creare le tabelle e, contestualmente, le classi che mappano a esse. La classe `Base`, che eredita da `DeclarativeBase`, è un artificio di SQLAlchemy che serve per ottenere l'istanza della classe `MetaData` che contiene le relazioni tra tabelle SQL e rappresentazione a oggetti. Ereditando da `Base`, i modelli popolano automaticamente l'istanza di `Metadata`.

L'oggetto `db` creato dalla libreria `Flask-SQLAlchemy` esponde due interfacce `db.Model`, per creare i modelli, e `db.session` per instaurare la connessione al database. `db.session` è già configurato dalla libreria come `scoped_session`, che è la versione thread-safe dell'oggetto `session` di SQLAlchemy: in altre parole, se la richiesta deve fare più accessi al database, non viene creata una nuova sessione per ogni accesso ma si riutilizza la stessa connessione.

Dopo aver creato le tabelle, la funzione `init_db()` attiva il vincolo di chiave esterna, che in SQLite è disabilitato di default per questioni di retrocompatibilità, e popola UTENTI e MERCATI con due record necessari per far funzionare il resto.

### Struttura del database

Usiamo tre tabelle:
- Utenti: per gestire le persone che possono usare l'applicazione e i relativi privilegi
- Corrispettivi: dove verranno conservati i dati inseriti
- Mercati: contiene informazioni sugli attuali mercati dell'azienda

La struttura delle tabelle è la seguente, dove una sottolineatura indica la chiave primaria e il corsivo indica una chiave esterna:

UTENTI(<u>username</u>, password, is_admin)  
CORRISPETTIVI(<u>ts</u>, *inserito_da*, data, cassa, *mercato*, *giorno_mercato*, reparto1, reparto2, reparto3, reparto4, reparto5)  
MERCATI(<u>nome, giorno</u>, is_evento, is_attuale)

CORRISPETTIVI(ts) è il timestamp del momento in cui viene registrato il corrispettivo. Il valore non è inserito dall'utente, ma compilato automaticamente dal web server.
MERCATI(giorno) è il giorno della settimana in cui si svolge il mercato registrato; Mercati(attuale), invece, indica se il mercato si svolge nel presente o se è stato dismesso.

### Vincoli

Tutti gli attributi hanno il vincolo NOT NULL, tranne CORRISPETTIVI(reparto1, reparto2, ..., reparto5) e Mercati(attuale).

Nella relazione CORRISPETTIVI, *inserito_da* è chiave esterna riferita a Utenti(user_name) e (*mercato*, *giorno_mercato*) è chiave esterna riferita a MERCATI.

Vincoli intrarelazionali su CORRISPETTIVI:
- il giorno della settimana di *data* deve essere uguale a *giorno_mercato*
- *data* non può essere una data successiva a quella indicata da *ts*

Vincoli intrarelazionali su MERCATO:
- Se *is_evento* è TRUE, allora *is_attuale* deve essere NULL; altrimenti *is_attuale* non può essere NULL
- Se il mercato non è attuale, il nome deve terminare con la stringa `_old`, es. "Centro_old"

  
Vincoli interrelazionali:
- Al momento dell'inserimento, il mercato referenziato da CORRISPETTIVI(mercato) deve essere un mercato attuale, a meno che non sia un mercato di tipo "evento"

Tutti i vincoli sono stati implementati a livello di DDL, tranne il primo vincolo su CORRISPETTIVI che deve essere implementato a livello di applicazione. Anche l'aggiunta automatica della stringa "_old" al nome di un mercato non più attuale non può essere gestita a livello di DBMS per limitazioni di SQLite sui trigger: gli inserimenti e le modifiche dei mercati non più attuali dovranno quindi essere opportunamente gestiti a livello di applicazione.

## Applicazione Flask

### Inizializzazione
Il file `__init__.py` funge da entry point per l'applicazione. Al suo interno, usando una factory function, crea e restituisce l'istanza dell'applicazione, facendo opportune configurazioni. L'opzione `instance_relative_config=True` serve a dire che i file per la configurazione sono relativi alla directory in cui si trova il package dell'applicazione. I file sono quindi esterni al package: in questo modo possiamo tenere separati i segreti e il database dal resto dell'applicazione.

### Blueprint
Modularizziamo l'applicazione raggruppando in tre blueprint le rotte. Tutte le rotte che assolvono funzionalità per l'autenticazione degli utenti sono raggruppate nel blueprint "auth", le rotte che permettono l'inserimento dei corrispettivi sono nel blueprint "corr" e, infine, le rotte per la gestione dei mercati sono nel blueprint "mercati".

### Autenticazione
I servizi di autenticazione sono tutti nel file `auth.py`.

L'autenticazione è gestita in maniera veramente basilare, confrontando se la password inserita dall'utente fa match con la password registrata nel database sotto lo stesso username. Interessante invece è l'uso dell'oggetto `session`, un oggetto speciale di Flask che consente di conservare informazioni tra tutte le richieste effettuate dallo stesso browser. Usiamo `session` per ricordare che l'utente è autenticato.

Per limitare l'accesso alle funzionalità dell'app ai soli utenti autenticati definiamo il decoratore `login_required()`. Il decoratore viene applicato sulle rotte alle quali vogliamo limitare l'accesso. Esso controlla se l'utente è attivo nella sessione, nel qual caso il flusso dell'esecuzione continua con la vista associata alla rotta; in caso contrario, effettua un redirect sulla rotta del login. Le sezioni dedicate solo all'admin sono invece protette dal decoratore `admin_required`.

Se l'utente non ha i privilegi di admin, dopo un'autenticazione terminata con successo, viene dirottato direttamente sulla pagina per l'inserimento dei corrispettivi. Se invece l'utente è un admin, viene dirottato in una pagina in cui può scegliere quale operazione effettuare.